In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import bokeh.layouts as bkl
import bokeh.plotting as bk
import bokeh.transform as btr
from bokeh.models import *
from bokeh.palettes import *

In [32]:
palette = lambda n: [(int(r), int(g), int(b), a) for r, g, b, a in (plt.get_cmap('rainbow')(np.linspace(0, 1, n)) * [255, 255, 255, 1])]

data = {
    'xy': {
        'data': np.random.rand(100,2),
        'umap': True,
    },
    'attr1': {
        'data': np.random.choice(['a', 'b', 'c'], 100),
        'color': True,
        'tooltip': True,
    },
    'attr2': {
        'data': np.random.randint(0, 100, n),
        'color': True,
        'tooltip': True,
    },
}

_color_attrs = [(key, value['data']) for key, value in data.items() if value.get('color')]
color_maps = {}
log=False
for key, value in _color_attrs:
    if isinstance(value[0], str):
        color_maps[key] = btr.factor_cmap(key, palette(len(list(set(value)))), list(set(value)))
    else:
        color_maps[key] = (btr.log_cmap if log else btr.linear_cmap)(key, palette(600), low=min(value), high=max(value))

embedding_df = pd.DataFrame()
for key, value in data.items():
    if value.get('umap'):
        embedding_df['x'], embedding_df['y'] = value['data'][:,0], value['data'][:,1]
    elif value.get('tooltip') or value.get('color'):
        embedding_df[key] = value['data']

_default_color_attr = list(color_maps.keys())[0]
_color_options = list(color_maps.keys())
coloringSelect = Select(
    title='Color by',
    value=_default_color_attr,
    options=_color_options
)
fig = bk.figure(
    title='Embedding of FRB 1D Time Series FFT (positive real part) with',
    tools=(
        CopyTool(),
        HoverTool(tooltips = "<div>" + "".join([f'<div><span style="font-size: 15px; font-weight: bold;">{attr_key}: @{attr_key}</span></div>' for attr_key, value in data.items() if value.get('tooltip')]) + "</div>"),
        LassoSelectTool(),
    )
)
plotPoints = fig.circle(
    x="x",
    y="y", 
    source=embedding_df,
    size=7,
    line_color='black',
    line_width=0.25,
    fill_alpha=0.7,
    fill_color=color_maps[_default_color_attr]
)
colorbar = plotPoints.construct_color_bar(padding=1)
fig.add_layout(colorbar, 'right')
coloringSelect.js_on_change('value', CustomJS(
    args=dict(plotPoints=plotPoints, color_maps=color_maps, colorbar=colorbar),
    code="""
    const selectedAttr = cb_obj.value;
    const colorMapper = color_maps[selectedAttr];
    
    plotPoints.glyph.fill_color = colorMapper;
    colorbar.color_mapper = colorMapper['transform'];
    """
))


fig.grid.visible = False
fig.axis.visible = False
plot = bkl.gridplot([[coloringSelect], [fig]],toolbar_location='below')
bk.output_notebook()
bk.show(plot)
bk.save(plot, 'test.html')

Loading BokehJS ...

/tmp/ipykernel_12576/4156594798.py:80: UserWarning: save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN
  bk.save(plot, 'test.html')
/tmp/ipykernel_12576/4156594798.py:80: UserWarning: save() called but no title was supplied and output_file(...) was never called, using default title 'Bokeh Plot'
  bk.save(plot, 'test.html')


'/home/guutz/dawn/test.html'

In [17]:
dict(field=_default_color_attr, transform=color_maps[_default_color_attr])

{'field': 'attr1',
 'transform': Field(field='attr1', transform=CategoricalColorMapper(id='p4211', ...), units=Unspecified)}

In [38]:
[][0]

IndexError: list index out of range